# Bert Topic modelling

## Running the example

First thing is to runn the example from https://maartengr.github.io/BERTopic/index.html to check everything is working. Output should match that shown in the Quickstart example on the website.

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

In [ ]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.get_document_info(docs)

## Running on London data

Now we know that berttopic is at least working as in the example, we can run it on the `london_urls.csv` data we prepared earlier.

In [ ]:
import pandas as pd

In [ ]:
london_df = pd.read_csv('london_urls.csv', usecols=['url', 'parent_url', 'content', 'name', 'keyword'])
london_df

In [ ]:
# get summary stats about the dataframe to check things look right.
london_df.describe()

### Approach 1: Train a model on all pages

In [ ]:
# fit a BERTopic model to the data and use it to find groups in its training data.
docs = list(london_df['content'])
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [ ]:
# look at a slice of the topics to see if they make sense. We see a lot of council names.
topic_model.get_topic_info()[40:60]

In [ ]:
# we can also get information about how documents relate to topics and why they've been assigned
topic_model.get_document_info(docs)

### Approach 2: Topic modelling with sites featuring keywords

In [ ]:
# make a new dataframe filtering to websites matching keywords
keyword_london_df = london_df[london_df['keyword']]
# we only want the documents for each site, not the whole dataframe
filtered_docs = list(keyword_london_df['content'].values)

In [ ]:
# train a new topic model just on websites matching keywords
filtered_topic_model = BERTopic()
f_topics, f_probs = filtered_topic_model.fit_transform(filtered_docs)

In [ ]:
# look at a slice of the topics again. These look more promising.
filtered_topic_model.get_topic_info()[1:30]

In [ ]:
# looking at the document info again
filtered_topic_model.get_document_info(filtered_docs)

In [ ]:
# export the enriched data for further processing
export_df = pd.concat([keyword_london_df.reset_index(drop=True), filtered_topic_model.get_document_info(filtered_docs)], axis=1)
export_df.to_csv("filtered_london_with_topics.csv")

In [ ]:
# filter to 100 topics and see what changes
filtered_topic_model.reduce_topics(filtered_docs, nr_topics=100)
filtered_topic_model.get_topic_info()

## Approach 3: Southwark only model

We found that the topic model tends to build topics from the councils. This makes sense based on how the model works, but we want it to build topics based on policy rather than location. We'll train a model on one council and see if it generalises. Southwark is the council with the most pages in the dataset so we'll use that.

In [ ]:
# southwark is the biggest
london_df['name'].value_counts()

In [ ]:
# filter to just southward and retrain the model
sdocs = keyword_london_df[keyword_london_df['name'].str.contains("Southwark")]['content']
s_topic_model = BERTopic()
s_topics, s_probs = s_topic_model.fit_transform(sdocs)

In [ ]:
# we don't see so many council names in the topics now
s_topic_model.get_topic_info()

In [ ]:
# we'll try reducing the number of topics and see how that affects robustness
s_topic_model.reduce_topics(sdocs, nr_topics=50)
s_topic_model.get_topic_info()

### Apply to other councils

In [ ]:
f_topics, f_probs = s_topic_model.transform(filtered_docs)

In [ ]:
keyword_london_df['Topic'] = f_topics
keyword_london_df['topic_prob'] = f_probs
keyword_london_df = london_df[london_df['keyword']]
keyword_london_df

In [ ]:
topics = s_topic_model.get_topic_info()[["Topic", "Name", "Representation"]]
topics = topics.set_index("Topic")
topics

In [ ]:
# look at topics, they look more reasonable
all_london_topics = keyword_london_df.join(topics, on="Topic")
all_london_topics["Representation"].value_counts()

In [ ]:
# export for further analysis
all_london_topics.to_csv("all_london_data_southwark_topics.csv")
topics.to_csv("southwark_topic_list.csv")